In [50]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from database_helper import *
from collections import Counter
from dateutil.relativedelta import relativedelta
from datetime import datetime

def prepare_data(city):
    """Fetch and prepare data for a given city from the SQLite database."""
    
    # Connect to the SQLite database
    conn = sqlite3.connect("validAddress.db")
    
    # Fetch data from the database
    results = fetch_city_from_db(conn, 'validAddress', city)
    
    # Determine the most common history length
    most_common_length = Counter(len(row[3]) for row in results).most_common(1)[0][0]
    
    # Create headers for the DataFrame
    data_date_dt = datetime.strptime(results[0][6], '%Y-%m-%d %H:%M:%S.%f')
    header = ['Address'] + [(data_date_dt - relativedelta(months=i)).strftime('%Y-%m') for i in range(most_common_length - 1, -1, -1)]
    
    # Transform results
    modified_results = [(address, *history) for id, address, city, history, status, status_date, data_date, beds, baths, year_built, sqft in results if len(history) == most_common_length]
    
    # Convert to DataFrame
    data = pd.DataFrame(modified_results, columns=header)
    
    conn.close()
    
    return data

In [51]:
def train_model(data,city):
    """Train a 1D CNN model on the provided data and return evaluation metrics and predictions."""
    
    # Extract addresses and prepare data
    addresses = data['Address']
    data = data.drop(columns=['Address'])
    X = data.values[:, :-1].reshape((-1, data.shape[1]-1, 1))
    y = data.values[:, -1]
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Define and compile the model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    # Train the model
    model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))
    
    # Evaluate and predict
    loss, mae = model.evaluate(X_test, y_test, verbose=0)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    predicted_next_month_prices = model.predict(X)
    
    # Prepare results
    predicted_prices_df = pd.DataFrame({
        'Address': addresses,
        'Predicted_Next_Month_Price': predicted_next_month_prices.flatten()
    })
    #model.save(f"saved_model/{city}-model")
    return mae, r2, predicted_prices_df


This is used to trial different vairables for ML model


In [52]:

def ml_setup(city):
    name=city.lower()
    data = prepare_data(name)
    mae, r2, predictions = train_model(data)
    print(mae,r2)
    
    
ml_setup("berlin")


Epoch 1/20
34/34 [==============================] - 1s 8ms/step - loss: 12786328576.0000 - mae: 77434.8594 - val_loss: 1337160192.0000 - val_mae: 29206.8438
Epoch 2/20
34/34 [==============================] - 0s 4ms/step - loss: 748081216.0000 - mae: 16984.6113 - val_loss: 594594880.0000 - val_mae: 15737.7256
Epoch 3/20
34/34 [==============================] - 0s 4ms/step - loss: 576699904.0000 - mae: 15072.4482 - val_loss: 415088416.0000 - val_mae: 12253.1855
Epoch 4/20
34/34 [==============================] - 0s 4ms/step - loss: 411784448.0000 - mae: 12487.2129 - val_loss: 365734208.0000 - val_mae: 12563.8057
Epoch 5/20
34/34 [==============================] - 0s 4ms/step - loss: 341862816.0000 - mae: 11568.8428 - val_loss: 262023024.0000 - val_mae: 9947.4678
Epoch 6/20
34/34 [==============================] - 0s 4ms/step - loss: 263249568.0000 - mae: 10219.6016 - val_loss: 218320576.0000 - val_mae: 8965.9844
Epoch 7/20
34/34 [==============================] - 0s 4ms/step - loss: 212